In [1]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.linalg import eig
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import matplotlib as mpl
import matplotlib.pyplot as plt
import particle_filters as pf

import pyhsmm
import pyhsmm.basic.distributions as distributions
from pyhsmm.util.general import rle

from pyhsmm.plugins.factorial.models import Factorial, FactorialComponentHSMM
from pyhsmm.internals.transitions import HDPHSMMTransitions
from pyhsmm.internals.initial_state import Uniform

import ast
import sys
import time

%matplotlib inline

In [2]:
theta = 20.
theta_out = 17.
lmbda = 0.5
gamma = 1.3
state = 1


In [3]:
def next_theta(theta,state,theta_out,lmbda,gamma):
    return theta - lmbda * (theta - theta_out) + gamma*state

In [4]:
theta_next = next_theta(20.,1,17.,0.5,1.3)
theta_next

19.8

In [5]:
theta_hat = theta
lmbda_hat = 0.6
gamma_hat = 1.1
state_hat = 1

In [6]:
theta_next_hat = next_theta(20.,1,17.,0.6,1.1)
theta_next_hat

19.3

In [7]:
np.random.binomial(size=20,p=0.5,n=1)

array([0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1])

In [9]:
def next_theta(theta,state,theta_out,lmbda,gamma):
    return theta - lmbda * (theta - theta_out) + gamma*state

def theta_seq(n,theta_start,states,theta_out,lmbda,gamma):
    assert len(states) == n-1
    seq = np.zeros(n)
    seq[0] = theta_start
    for i in np.arange(1,n):
        seq[i] = next_theta(seq[i-1],states[i-1],theta_out,lmbda,gamma)
    return seq

def gradient_descent(n,theta_start,theta_out,states,lmbda,gamma,epsilon,max_iter):
    l = 0.6
    g = 1.1
    alpha = 0.01
    N = 0
    
    theta_true_seq = theta_seq(n,theta_start,states,theta_out,lmbda,gamma)
    theta_true = theta_true_seq[n-1]
    theta_hat_seq = theta_seq(n,theta_start,states,theta_out,l,g)
    error = theta_hat_seq[n-1] - theta_true
    end_test = np.max(np.abs(theta_hat_seq - theta_true_seq))
    
    while end_test>=epsilon and N<max_iter:
        
        #l = l - alpha * -2.*(theta_start - theta_out)*error
        l = l - alpha * 2.*error*((theta_out - theta_start)*np.power(1.-l,n-1)-np.sum(theta_hat_seq[:n-1]*np.power(1.-l,np.arange(n-1,0,-1))))
        g = g - alpha * 2.*error*np.sum(states)
        
        theta_hat_seq = theta_seq(n,theta_start,states,theta_out,l,g)
        error = theta_hat_seq[n-1] - theta_true
        end_test = np.max(np.abs(theta_hat_seq - theta_true_seq))
        
        N += 1
    return l, g, theta_hat_seq,end_test,N

In [10]:
states = np.random.binomial(size=19,p=0.5,n=1)
theta_seq(20,20.,states,17.,0.5,1.3)

array([ 20.        ,  19.8       ,  18.4       ,  19.        ,
        18.        ,  17.5       ,  17.25      ,  18.425     ,
        19.0125    ,  19.30625   ,  19.453125  ,  18.2265625 ,
        18.91328125,  17.95664062,  18.77832031,  19.18916016,
        18.09458008,  18.84729004,  19.22364502,  18.11182251])

In [12]:
gradient_descent(20,20.,17.,states,0.5,1.3,0.001,10000)

(0.4785508669874759,
 1.2060205014338965,
 array([ 20.        ,  19.7703679 ,  18.44460594,  18.95930902,
         18.02167999,  17.53275414,  17.27780419,  18.35088125,
         18.91043636,  19.20221588,  19.35436407,  18.2276811 ,
         18.84619375,  17.96269613,  18.70801756,  19.09666478,
         18.09330403,  18.77612294,  19.13217827,  18.11182251]),
 0.10403411500725923,
 10000)